In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from matplotlib import cm, ticker
from sklearn.metrics import classification_report
from tqdm import tqdm

from counterfactuals.datasets.moons import MoonsDataset
from counterfactuals.optimizers.approach_gen_disc_loss import ApproachGenDiscLoss
from counterfactuals.losses import BinaryDiscLoss

from counterfactuals.utils import plot_loss_space

In [ ]:
def plot_model_distribution(model, median_prob=None, disc_model=None):
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(20,12)

    xline = torch.linspace(-1.5, 2.5, 200)
    yline = torch.linspace(-.75, 1.25, 200)
    xgrid, ygrid = torch.meshgrid(xline, yline)
    xyinput = torch.cat([xgrid.reshape(-1, 1), ygrid.reshape(-1, 1)], dim=1)
    
    with torch.no_grad():
        zgrid0 = model.log_prob(xyinput, torch.zeros(40000, 1)).exp().reshape(200, 200)
        zgrid1 = model.log_prob(xyinput, torch.ones(40000, 1)).exp().reshape(200, 200)

    zgrid0 = zgrid0.numpy()
    zgrid1 = zgrid1.numpy()

from counterfactuals.optimizers.approach_three import ApproachThree
from counterfactuals.optimizers.ppcef import PPCEF
from counterfactuals.losses import BinaryDiscLoss

    
    cs0 = ax.contour(xgrid.numpy(), ygrid.numpy(), zgrid0, levels=10, cmap='Greys', linewidths=0.4, antialiased=True)
    cs1 = ax.contour(xgrid.numpy(), ygrid.numpy(), zgrid1, levels=10, cmap='Oranges', linewidths=0.4, antialiased=True)
    return ax

In [ ]:
dataset = MoonsDataset(file_path="../data/moons.csv")

disc_model = torch.load("../models/disc_model_LR_MoonsDataset.pt")
flow = torch.load("../models/gen_model_FLOW_orig_MoonsDataset.pt")
cf = ApproachGenDiscLoss(
    gen_model=flow,
    disc_model=disc_model,
    disc_model_criterion=BinaryDiscLoss(),
    checkpoint_path="model.pt",
    neptune_run=None,
)

In [ ]:
median_prob = cf.calculate_median_log_prob(dataset.train_dataloader(batch_size=64, shuffle=False))

x_origin = torch.tensor([[-0.6,  0.65]], requires_grad=False)
x = torch.tensor([[-0.6,  0.65]], requires_grad=True)
context_origin = torch.Tensor([0])
context_target = torch.Tensor([1])

optimizer = torch.optim.Adam([x], lr=0.005)
min_loss = np.inf
no_improve = 0

num_iterations = 2000
patience = 20
alpha = 10


p_hist = []

for i in tqdm(range(num_iterations)):
    optimizer.zero_grad()
    loss_components = cf.search_step(x, x_origin, context_origin, context_target, alpha=alpha, delta=median_prob)
    mean_loss = loss_components["loss"].mean()
    mean_loss.backward()
    optimizer.step()

    if mean_loss.item() < min_loss:
        min_loss = mean_loss.item()
    else:
        no_improve += 1
    if no_improve > patience:
        break

    if i % 150 == 0:
        p_hist.append(x.detach().numpy().copy())

p_hist.append(x.detach().numpy().copy())
p_hist = np.concatenate(p_hist)


In [ ]:
flow = MaskedAutoregressiveFlow(features=2, hidden_features=4, context_features=1)
cf = PPCEF(
    gen_model=flow,
    disc_model=disc_model,
    disc_model_criterion=torch.nn.BCELoss(),
    checkpoint_path="model.pt",
    neptune_run=None,
)

In [ ]:
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True, noise_lvl=1e-5)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=200,
)

In [ ]:
disc_model(x_origin), disc_model(x)

In [ ]:
## Distribution Plot
ax = plot_model_distribution(cf.gen_model, median_prob, disc_model)

## Classifier Line
w1, w2 = list(disc_model.parameters())[0].detach().cpu().numpy()[0]
b = list(disc_model.parameters())[1].detach().cpu().numpy().item()
c = -b/w2
m = -w1/w2
xmin, xmax = -1.5, 2.5
ymin, ymax = -1.5, 2.5
xd = np.array([xmin, xmax])
yd = m*xd + c
plt.plot(xd, yd, '#ADD8E6', lw=2.0, ls="dashed")
plt.axis("off")

## Arrows
for i in range(p_hist.shape[0]-1):
    ax.arrow(p_hist[i, 0], p_hist[i, 1], p_hist[i+1, 0]-p_hist[i, 0], p_hist[i+1, 1]-p_hist[i, 1], width=0.015, lw=0.001, length_includes_head=True, alpha=0.5, color="k")

ax.scatter(p_hist[0:1, 0], p_hist[0:1, 1], c="k", s=100, alpha=0.8)
ax.scatter(p_hist[1:-1, 0], p_hist[1:-1, 1], c="orange", s=100, alpha=0.8)
ax.scatter(p_hist[-1:, 0], p_hist[-1:, 1], c="#DC143C", s=100, alpha=0.8)
_ = ax.axis("off")

## Save Figure
plt.tight_layout()
plt.savefig('moons.pdf')